# Exercise 3: Durable Agent — Solution

Complete implementation combining OpenAI agents with Temporal for durability.

## Prerequisites

Ensure the Temporal dev server is running and `.env` contains your OpenAI credentials.

In [ ]:
%pip install --quiet temporalio openai rich

import os
import asyncio
import uuid
from datetime import timedelta
from openai import OpenAI
from rich.console import Console
from temporalio import activity, workflow
from temporalio.client import Client
from temporalio.worker import Worker
from temporalio.common import RetryPolicy

console = Console()


def get_weather(location: str) -> str:

    weather_data = {        return result

        "San Francisco": "sunny, 72°F",        workflow.logger.info("✅ Workflow completed successfully")

        "New York": "cloudy, 65°F",

        "London": "rainy, 58°F",        )

        "Tokyo": "clear, 70°F",            ),

    }                backoff_coefficient=2.0,

    weather = weather_data.get(location, "partly cloudy, 68°F")                maximum_attempts=3,

    return f"The weather in {location} is {weather}"                maximum_interval=timedelta(seconds=10),

                initial_interval=timedelta(seconds=1),

            retry_policy=RetryPolicy(

@activity.defn            start_to_close_timeout=timedelta(seconds=30),

async def call_agent_with_tools(query: str, trace_id: str) -> str:            args=[query, trace_id],

    activity.logger.info("🤖 Activity started")            call_agent_with_tools,

    activity.logger.info(f"   Query: {query}")        result = await workflow.execute_activity(

    activity.logger.info(f"   Trace ID: {trace_id}")

        workflow.logger.info(f"   Trace ID: {trace_id}")

    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))        workflow.logger.info(f"   Query: {query}")

        workflow.logger.info("🚀 Durable agent workflow started")

    tools = [    async def run(self, query: str, trace_id: str) -> str:

        {    @workflow.run

            "type": "function",class DurableAgentWorkflow:

            "function": {@workflow.defn

                "name": "get_weather",

                "description": "Get the current weather for a location",

                "parameters": {    return final_message

                    "type": "object",    activity.logger.info("✅ Activity completed")

                    "properties": {

                        "location": {        final_message = response_message.content

                            "type": "string",    else:

                            "description": "The city name",        final_message = final_response.choices[0].message.content

                        }        )

                    },            messages=messages,

                    "required": ["location"],            model="gpt-4o-mini",

                },        final_response = client.chat.completions.create(

            },

        }                )

    ]                    }

                        "content": function_response,

    messages = [{"role": "user", "content": query}]                        "name": function_name,

    response = client.chat.completions.create(                        "role": "tool",

        model="gpt-4o-mini",                        "tool_call_id": tool_call.id,

        messages=messages,                    {

        tools=tools,                messages.append(

    )                function_response = get_weather(**function_args)

            if function_name == "get_weather":

    response_message = response.choices[0].message

    tool_calls = response_message.tool_calls            activity.logger.info(f"   Calling: {function_name}({function_args})")

            function_args = eval(tool_call.function.arguments)

    if tool_calls:            function_name = tool_call.function.name

        activity.logger.info(f"🔧 Tool calls detected: {len(tool_calls)}")        for tool_call in tool_calls:

        messages.append(response_message)


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Run the Solution

Execute the workflow and observe durable behavior with retries.

In [2]:
# Run the durable agent solution
async def run_solution() -> None:
    console.print("\n[bold cyan]🚀 Exercise 3: Durable Agent — Solution[/bold cyan]\n")

    trace_id = str(uuid.uuid4())
    console.print(f"[yellow]Trace ID:[/yellow] {trace_id}\n")

    client = await Client.connect("localhost:7233")
    task_queue = "durable-agent-queue"

    async with Worker(
        client,
        task_queue=task_queue,
        workflows=[DurableAgentWorkflow],
        activities=[call_agent_with_tools],
    ):
        query = "What's the weather like in San Francisco?"
        workflow_id = f"durable-agent-{trace_id}"

        console.print(f"[yellow]Query:[/yellow] {query}\n")

        result = await client.execute_workflow(
            DurableAgentWorkflow.run,
            args=[query, trace_id],
            id=workflow_id,
            task_queue=task_queue,
        )

    console.print(f"\n[bold green]🤖 Agent Response:[/bold green]\n{result}\n")
    console.print(
        f"[yellow]View in Temporal UI:[/yellow] "
        f"http://localhost:8233/namespaces/default/workflows/{workflow_id}"
    )
    console.print(f"[yellow]Trace ID for correlation:[/yellow] {trace_id}\n")

asyncio.run(run_solution())


RuntimeError: asyncio.run() cannot be called from a running event loop

## Source Code

The workflow and activity implementations are defined above in the first code cell.